# Add Plugins Step by Step - using PlugnPlai and LangChain

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edreisMD/plugnplai/blob/main/examples/plugins_step_by_step.ipynb)

The goal of this example is to go through all the steps to add plugins to LLMs
1. Get plugins of certain categories from [plugnplai.com](https://plugnplai.com)
2. Load plugins manifest and specifications
3. Parse specifications and generate a prompt with the descriptions
4. Use [LangChain]() to call the LLM
5. Parse the LLM response, looking for the `[API]` pattern defined on `plugins.prompt`
6. Call the plugin using `plugins.call()`
7. Use LangChain again to ask the LLM a final response using the new data

# Install

In [8]:
pip uninstall plugnplai -y

# Get the plugins from the directory

We want to install three plugins at maximum to fit the description on the context length

Lets find one plugin for each category:
1. travel
2. shopping 
3. weather

We can use PlugnPlai categories (see [API reference](https://plugnplai.github.io/))

In [2]:
import plugnplai as pl

# Get working plugins - only tested plugins (in progress)
urlsTravel = pl.get_plugins(category='travel')["urls"]
print(f'Travel plugins: {urlsTravel}')

urlsShopping = pl.get_plugins(category='shopping')["urls"]
print(f'Shopping plugins: {urlsShopping}')

urlsLan = pl.get_plugins(category='language')["urls"]
print(f'Weather plugins: {urlsLan}')

Travel plugins: ['https://gogaffl.com', 'https://trip.com', 'https://api.yelp.com', 'https://gps-telecom.com']
Shopping plugins: ['https://pricerunner.com', 'https://server.shop.app', 'https://klarna.com']
Weather plugins: ['https://api.tomorrow.io']


In [3]:
# Lets pick one of each list and add to our url list

urls = []

# Trip (list index 1)
urls.append(urlsTravel[1])

# Klarna (list index 2)
urls.append(urlsShopping[2])

# Speak (list index 0)
urls.append(urlsLan[0])

print(f'Our chosen Plugins: {urls}')

Our chosen Plugins: ['https://trip.com', 'https://klarna.com', 'https://api.tomorrow.io']


# Load and activate the plugins

In [4]:
from plugnplai import Plugins

plugins = Plugins.install_and_activate(urls)

## Print the default prompt for the active plugins

In [5]:
print(plugins.prompt)


# SYSTEM MESSAGE
You are a large language model trained to assist humans.
Knowledge Cutoff: 2021-09
Current date: 2023-05-12
Below is a list of available APIs that you can utilize to fulfill user requests. 
When using an API, please follow the specified format to make the API call. 
If possible, avoid asking follow-up questions and aim to complete the task with the information provided by the user.

To make an API call, use the following format:

[API]namespace.operationId[/API]
[PARAMS]{ 
    "parameter_name": "parameter_value",
    ...
}[/PARAMS]

For example, to call an API operation with the operation ID "productsUsingGET" in the "KlarnaProducts" namespace, 
and provide the required parameters "q" and "size", the format would be as follows:

[API]KlarnaProducts.productsUsingGET[/API]
[PARAMS]{
    "q": "t-shirt", 
    "size": 3
}[/PARAMS]

Please ensure that you use the correct namespace and operation ID, and provide the necessary parameters for each API call. 
After requesting th

## Lets look at the length of the prompt

Get the number of tokens of the prompt by just calling 'plugins.tokens' 

In [6]:
print(plugins.tokens)

731


## Call the LLM using LangChain

# You will need to first define your API key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate,
from langchain.schema import AIMessage, HumanMessage, SystemMessage

#### Uncomment or modify the message to test different plugins

In [ ]:
# Test Klarna Plugin
# HUMAN_MESSAGE = "I want to buy a rolling stones t-shirt"

# Test Trip Plugin
HUMAN_MESSAGE = "I need a hotel in Paris next weekend."

# Test Speak Plugin
# HUMAN_MESSAGE = "How to say I love you in Portuguese?"

#### Call LLM

In [1]:
chat = ChatOpenAI(temperature=0, model="gpt-4")

messages = [
    SystemMessage(content=plugins.prompt),
    HumanMessage(content=HUMAN_MESSAGE)
]

# add_plugins = AddPlugins(plugins)

# chatWithPlugins = add_plugins(chat)
# res = chatWithPlugins(messages)

res = chat(messages)

llm_first_response = res.content

print(llm_first_response)

NameError: name 'ChatOpenAI' is not defined

## Parse the LLM response

In [ ]:
# import the parser function
from pluginplai import parse_llm_response

# Parse the LLM response importing '
call_dict = parse_llm_response(llm_first_response)
print(call_dict)

## Call the API

In [2]:
r = plugins.call_api(plugin_name = call_dict['plugin_name'], 
                    operation_id = call_dict['operation_id'], 
                    parameters = call_dict['parameters']
                    )

api_response = r.json()
r.json()

NameError: name 'plugins' is not defined

## LLM responds using the API data

In [3]:
api_return_prompt = f"""
Assistant is a large language model with access to plugins.

Assistant called a plugin in response to this human message:
# HUMAN MESSAGE
{HUMAN_MESSAGE}

# API REQUEST SUMMARY
{llm_first_response}

# API RESPONSE
{api_response}
"""

# Install the plugins ewith the original template
plugins = Plugins.install_and_activate(urls)

chat = ChatOpenAI(temperature=0, model="gpt-4")
# chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

messages = [
    SystemMessage(content=api_return_prompt),
    HumanMessage(content="HUMAN_MESSAGE")
]

res = chat(messages)

print(res.content)

NameError: name 'HUMAN_MESSAGE' is not defined

# Lets copy the response to a markdown cell:

I found 3 hotels in Paris for next weekend (May 19-21, 2023). Here are the top options:

1. [Le Tsuba Hotel](https://us.trip.com/hotels/detail/?cityId=192&hotelId=6597288&checkin=2023-05-19&checkout=2023-05-21&curr=USD)
   - Address: 45 Rue des Acacias
   - Price: $267 USD
   - Star rating: 4 stars
   - Score: 4.6/5.0 (36 reviews)
   - Features: Sauna, fitness room

2. [Pullman Paris Centre - Bercy](https://us.trip.com/hotels/detail/?cityId=192&hotelId=2107175&checkin=2023-05-19&checkout=2023-05-21&curr=USD)
   - Address: 1 Rue de Libourne
   - Price: $337 USD
   - Star rating: 4 stars
   - Score: 4.5/5.0 (42 reviews)
   - Features: Swimming pool, children's playground

3. [Shangri-La Paris](https://us.trip.com/hotels/detail/?cityId=192&hotelId=730333&checkin=2023-05-19&checkout=2023-05-21&curr=USD)
   - Address: 10 Av. d'Iéna
   - Price: $2074 USD
   - Star rating: 5 stars
   - Score: 4.6/5.0 (6 reviews)
   - Features: Sauna, swimming pool

Please note that prices and availability are subject to change. Make sure to book your preferred hotel as soon as possible.